# **Full GPU-Accelerated Variational AutoEncoder Implementation in CUDA**

## Experimental Environment Setup

In this section we prepare and validate the experimental environment used
for all subsequent benchmarks and analyses.

We first verify that a CUDA-capable GPU is available and that the CUDA compiler (`nvcc`) is correctly installed.  
This step ensures that the benchmarks will run on the expected hardware.

In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
!apt-get update -y
!apt-get install -y nsight-systems-2025.5.2

We clone the project repository from GitHub and place it in the working directory.
This step recreates the exact codebase used for the experiments.

In [ ]:
REPO_URL="https://github.com/massimo-ruggiero/vae-cuda"
PROJECT_DIR="VAE"

In [ ]:
%cd /content
!rm -rf "$PROJECT_DIR"
!git clone --depth 1 "$REPO_URL" "$PROJECT_DIR"
%cd "$PROJECT_DIR"

We inspect the directory structure of the repository to verify that all expected
modules and scripts are present.

In [ ]:
!sudo apt-get update -y >/dev/null
!sudo apt-get install -y tree >/dev/null
!tree -L 4

The repository includes helper scripts for running the main training pipeline, the *micro* and *macro* benchmark suite.

In [ ]:
!ls -la scripts

The VAE implementation expects the MNIST dataset to be provided in a custom
binary format for fast loading during training and benchmarking.

In [ ]:
import os
import numpy as np
from tensorflow.keras.datasets import mnist

def save_to_bin(images, labels, filename):
    images_flat = images.reshape(images.shape[0], -1).astype(np.uint8)
    labels = labels.astype(np.uint8)

    num_samples = images.shape[0]

    header = np.array([num_samples], dtype=np.int32)

    print(f"Scrittura {filename}...")
    print(f"  - Samples: {num_samples}")
    print(f"  - Dimensioni Dati: {images_flat.shape}")
    print(f"  - Dimensioni Labels: {labels.shape}")

    with open(filename, 'wb') as f:
        header.tofile(f)
        images_flat.tofile(f)
        labels.tofile(f)

    size_mb = os.path.getsize(filename) / (1024 * 1024)
    print(f"  -> Completato! ({size_mb:.2f} MB)\n")


if not os.path.exists('data'):
    os.makedirs('data')
    print("Cartella 'data/' creata.")

print("Scaricamento MNIST da Keras...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Genera i file binari
save_to_bin(x_train, y_train, 'data/train.bin')
save_to_bin(x_test, y_test,  'data/test.bin')

print("Tutto fatto. Ora puoi lanciare il programma C++.")


## End-to-End Sanity Check


Before running the full benchmark suite, we perform a quick end-to-end test to verify that:
- the project compiles and runs correctly on the current GPU
- training executes without runtime errors
- the VAE produces a valid reconstruction
- the sampling pipeline generates plausible outputs

This step is not meant to optimize performance: it is a correctness + pipeline validation check.

In [ ]:
!chmod +x scripts/run_sanity_check.sh
!bash scripts/run_sanity_check.sh

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


IMG_SIZE = 28
IMG_PIXELS = IMG_SIZE * IMG_SIZE


def load_raw_image(path: str) -> np.ndarray:
    data = np.fromfile(path, dtype=np.float32)

    if data.size != IMG_PIXELS:
        raise ValueError(
            f"{path}: expected {IMG_PIXELS} values, found {data.size}"
        )

    return data.reshape(IMG_SIZE, IMG_SIZE)


def show_reconstruction(original_path: str, reconstructed_path: str):
    img_orig = load_raw_image(original_path)
    img_recon = load_raw_image(reconstructed_path)

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title("Original input")
    plt.imshow(img_orig, cmap="gray", vmin=0, vmax=1)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("VAE reconstruction")
    plt.imshow(img_recon, cmap="gray", vmin=0, vmax=1)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


def show_sample(sample_path: str, title: str = "VAE sample"):
    img = load_raw_image(sample_path)

    plt.figure(figsize=(4, 4))
    plt.title(title)
    plt.imshow(img, cmap="gray", vmin=0, vmax=1)
    plt.axis("off")
    plt.show()

In [ ]:
print("📂 Loading raw images...")

try:
    # --- paths ---
    base_dir = Path("images")
    original = base_dir / "original.raw"
    reconstructed = base_dir / "reconstructed.raw"

    sample_0 = base_dir / "sample_0.raw"

    # --- visualizations ---
    show_reconstruction(original, reconstructed)
    show_sample(sample_0, title="VAE sample")

except FileNotFoundError as e:
    print("❌ File not found:", e)
    print("Make sure you have run the 'run_sanity_check.sh' file first.")
except ValueError as e:
    print("❌ Data error:", e)

## Micro-Benchmark Suite Execution

After validating the end-to-end execution of the VAE pipeline, we run a dedicated
micro-benchmark suite to evaluate the performance of individual CUDA kernels.

The micro-benchmark script supports a configurable output directory.

- **Default output directory:** `results/`
- **Custom output directory:** specified via the `--outdir <path>` option

All benchmark results are stored as CSV files inside the selected directory.

In [ ]:
!chmod +x scripts/run_micro_bench.sh
!bash scripts/run_micro_bench.sh

In [ ]:
!chmod +x scripts/run_ncu_profiling.sh
!bash scripts/run_ncu_profiling.sh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams.update({
    "font.size": 13,            # default per tutto
    "axes.labelsize": 14,       # etichette assi (Time (ms), ecc.)
    "axes.titlesize": 14,       # titolo (se lo usi)
    "xtick.labelsize": 13,      # numeri e label asse X
    "ytick.labelsize": 13,      # numeri asse Y
    "legend.fontsize": 12,      # voci legenda
    "legend.title_fontsize": 13 # titolo legenda
})

def plot_all_operations_by_strategy(
    csv_path: str,
    output_dir: str,
    time_col: str = "median_ms",
    err_col: str = "mad_ms",
    strategy_col: str = "strategy",
    op_col: str = "operation",
    figsize=(15, 5),
):
    df = pd.read_csv(csv_path, comment="#")

    required = {op_col, strategy_col, time_col, err_col}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}. Found: {list(df.columns)}")

    df = df.dropna(subset=[op_col, strategy_col, time_col, err_col]).copy()
    if df.empty:
        raise ValueError("CSV has no valid rows to plot.")

    # Strategy order = appearance order in the CSV
    strategy_order = df[strategy_col].astype(str).drop_duplicates().tolist()
    operations = df[op_col].astype(str).drop_duplicates().tolist()

    # Handle possible duplicates defensively
    key = [op_col, strategy_col]
    has_dups = df.duplicated(subset=key, keep=False).any()

    if has_dups:
        data = (
            df.groupby(key, as_index=False)
              .agg(
                  time=(time_col, "mean"),
                  err=(err_col, "mean"),
                  n=(time_col, "size")
              )
        )
    else:
        data = (
            df.rename(columns={time_col: "time", err_col: "err"})
              [[op_col, strategy_col, "time", "err"]]
              .copy()
        )
        data["n"] = 1

    # ---- Plot
    fig, ax = plt.subplots(figsize=figsize)
    x = np.arange(len(strategy_order))

    group_width = 0.85
    bar_width = group_width / max(1, len(operations))

    y_max = (data["time"] + data["err"]).max()
    text_offset = max(0.01 * y_max, 0.3)

    for j, op in enumerate(operations):
        sub = data[data[op_col].astype(str) == op].set_index(strategy_col)

        present_strats = [s for s in strategy_order if s in sub.index.astype(str)]
        if not present_strats:
            continue

        idx = [strategy_order.index(s) for s in present_strats]
        times = sub.loc[present_strats, "time"].to_numpy(dtype=float)
        errs = sub.loc[present_strats, "err"].to_numpy(dtype=float)

        offset = (j - (len(operations) - 1) / 2) * bar_width
        xpos = np.array(idx, dtype=float) + offset

        ax.bar(
            xpos,
            times,
            width=bar_width * 0.95,
            yerr=errs,
            capsize=5,
            label=str(op),
        )

        for xx, t, e in zip(xpos, times, errs):
            ax.text(xx, t + e + text_offset, f"{t:.2f}", ha="center", va="bottom", fontsize=10.5)

    # Pretty x labels
    pretty_labels = [s.replace("_", " ") for s in strategy_order]
    ax.set_xticks(x)
    ax.set_xticklabels(pretty_labels, rotation=25, ha="right")

    ax.set_ylabel("Time (ms)")
    ax.grid(axis="y", linestyle="--", alpha=0.6)
    ax.legend(title="Operation")

    # Extra headroom
    top_needed = (data["time"] + data["err"]).max() + text_offset
    ax.set_ylim(0, top_needed * 1.12)

    plt.tight_layout()

    # ---- Save
    #out_dir = Path(output_dir)
    #out_dir.mkdir(parents=True, exist_ok=True)
    #
    #csv_name = Path(csv_path).stem
    #out_path = out_dir / f"{csv_name}.pdf"
    #
    #plt.savefig(out_path, format="pdf", bbox_inches="tight")
    #plt.close(fig)

    plt.show()


In [ ]:
plot_all_operations_by_strategy("/content/VAE/results/micro_bench/csv/bench_linalg.csv",
                                output_dir="/content/VAE/results/micro_bench/plots")

In [ ]:
from pathlib import Path
import pandas as pd


def best_kernels(
    csv_dir: str,
    time_col: str = "median_ms",
):
    """
    Per ogni CSV nella cartella:
      - per ogni 'operation' prende la riga con tempo minimo (time_col)
    Restituisce un unico DataFrame con:
      source_file, operation, strategy, median_ms
    """
    p = Path(csv_dir)
    files = sorted(p.glob("*.csv")) if p.is_dir() else [p]

    files = [f for f in files if f.exists() and f.suffix.lower() == ".csv"]
    if not files:
        raise FileNotFoundError(f"No CSV files found in: {csv_dir}")

    out = []
    for f in files:
        df = pd.read_csv(f, comment="#")

        required = {"operation", "strategy", time_col}
        missing = required - set(df.columns)
        if missing:
            raise ValueError(
                f"Missing columns in {f.name}: {missing}. Found: {list(df.columns)}"
            )

        df = df.dropna(subset=["operation", "strategy", time_col]).copy()
        df[time_col] = df[time_col].astype(float)
        df["source_file"] = f.name

        # best (min time) per operation, SOLO dentro questo file
        best_in_file = (
            df.sort_values(["operation", time_col], ascending=[True, True])
              .drop_duplicates(subset=["source_file", "operation"], keep="first")
              [["source_file", "operation", "strategy", time_col]]
              .rename(columns={time_col: "median_ms"})
              .reset_index(drop=True)
        )

        out.append(best_in_file)

    result = pd.concat(out, ignore_index=True)
    display(result)
    return result


In [ ]:
best_df = best_kernels("/content/VAE/results/micro_bench/csv")

## Macro-Benchmatk Suite Execution

### PyTorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

In [ ]:
PATH = "~/dataset"
BATCH_SIZE = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = MNIST(root=PATH, train=True, download=True, transform=mnist_transform)
test_dataset = MNIST(root=PATH, train=False, download=True, transform=mnist_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class VAE(nn.Module):
  def __init__(self, input_dim, hidden_dim, latent_dim):
    super(VAE, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.latent_dim = latent_dim

    # encoder
    self.encoder = nn.Sequential(
        nn.Linear(input_dim, hidden_dim),
        nn.LeakyReLU(0.2),
        nn.Linear(hidden_dim, hidden_dim),
        nn.LeakyReLU(0.2)
    )
    self.fc_mean = nn.Linear(hidden_dim, latent_dim)
    self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    # decoder
    self.decoder = nn.Sequential(
        nn.Linear(latent_dim, hidden_dim),
        nn.LeakyReLU(0.2),
        nn.Linear(hidden_dim, hidden_dim),
        nn.LeakyReLU(0.2),
        nn.Linear(hidden_dim, input_dim),
        nn.Sigmoid()
    )

  def encode(self, x):
    x = self.encoder(x)
    mean, logvar = self.fc_mean(x), self.fc_logvar(x)
    return mean, logvar

  def reparameterize(self, mean, var):
    epsilon = torch.randn_like(var).to(DEVICE)
    z = mean + var * epsilon
    return z

  def decode(self, x):
    return self.decoder(x)

  def forward(self, x):
    mean, logvar = self.encode(x)
    z = self.reparameterize(mean, torch.exp(0.5 * logvar))
    x_hat = self.decode(z)
    return x_hat, mean, logvar

  def generate_digit(self):
    noise = torch.randn(BATCH_SIZE, self.latent_dim).to(DEVICE)
    return self.decoder(noise)

In [ ]:
import time
import statistics

def loss_function(x, x_hat, mean, logvar):
  reconstruction_loss = F.binary_cross_entropy(x_hat, x, reduction='sum')
  KLD = - 0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
  return reconstruction_loss + KLD

def train(model, optimizer, epochs, x_dim):
  model.train()
  epoch_times = []
  total_start_time = time.perf_counter()

  for epoch in range(epochs):
      epoch_start_time = time.perf_counter()
      overall_loss = 0

      for batch_idx, (x, _) in enumerate(train_loader):
          x = x.view(-1, x_dim).to(DEVICE)
          optimizer.zero_grad()
          x_hat, mean, logvar = model(x)
          loss = loss_function(x, x_hat, mean, logvar)
          overall_loss += loss.item()
          loss.backward()
          optimizer.step()

      epoch_end_time = time.perf_counter()
      epoch_duration = epoch_end_time - epoch_start_time
      epoch_times.append(epoch_duration)

      avg_loss = overall_loss / (len(train_loader.dataset))
      print(f"Epoch {epoch + 1} | Average Loss: {avg_loss:.4f} | Time: {epoch_duration:.2f}s")

  total_end_time = time.perf_counter()
  total_duration = total_end_time - total_start_time
  median_time = statistics.median(epoch_times)

  print("-" * 30)
  print(f"Median time per epoch: {median_time:.4f}s")
  print(f"Total training time: {total_duration:.2f}s")

  return overall_loss

In [ ]:
model = VAE(input_dim=784, hidden_dim=400, latent_dim=200).to(DEVICE)
optimizer = Adam(model.parameters(), lr=1e-3)
train(model, optimizer, 100, 784)

### TensorFlow Implementation

In [ ]:
import os
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import ops
from keras import layers

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_digits = mnist_digits.reshape(-1, 784)

In [ ]:
class Sampling(layers.Layer):
  def __init__(self, **kwargs):
      super().__init__(**kwargs)
      self.seed_generator = keras.random.SeedGenerator(1337)

  def call(self, inputs):
      z_mean, z_log_var = inputs
      batch = ops.shape(z_mean)[0]
      dim = ops.shape(z_mean)[1]
      epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
      return z_mean + ops.exp(0.5 * z_log_var) * epsilon

In [ ]:
input_dim = 784
hidden_dim = 400
latent_dim = 200

# Encoder
encoder_inputs = keras.Input(shape=(input_dim,))
x = layers.Dense(hidden_dim)(encoder_inputs)
x = layers.LeakyReLU(negative_slope=0.2)(x)
x = layers.Dense(hidden_dim)(x)
x = layers.LeakyReLU(negative_slope=0.2)(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(hidden_dim)(latent_inputs)
x = layers.LeakyReLU(negative_slope=0.2)(x)
x = layers.Dense(hidden_dim)(x)
x = layers.LeakyReLU(negative_slope=0.2)(x)
decoder_outputs = layers.Dense(input_dim, activation="sigmoid")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
      with tf.GradientTape() as tape:
          z_mean, z_log_var, z = self.encoder(data)
          reconstruction = self.decoder(z)

          bce = keras.losses.binary_crossentropy(data, reconstruction)
          reconstruction_loss = ops.mean(ops.sum(bce))

          kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
          kl_loss = ops.mean(ops.sum(kl_loss, axis=1))

          total_loss = reconstruction_loss + kl_loss

      grads = tape.gradient(total_loss, self.trainable_weights)
      self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

      self.total_loss_tracker.update_state(total_loss)
      self.reconstruction_loss_tracker.update_state(reconstruction_loss)
      self.kl_loss_tracker.update_state(kl_loss)
      return {
          "loss": self.total_loss_tracker.result(),
          "reconstruction_loss": self.reconstruction_loss_tracker.result(),
          "kl_loss": self.kl_loss_tracker.result(),
      }

In [ ]:
import time
import statistics

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_times = []
        self.total_start_time = time.perf_counter()

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.perf_counter()

    def on_epoch_end(self, epoch, logs=None):
        duration = time.perf_counter() - self.epoch_start_time
        self.epoch_times.append(duration)
        print(f" - epoch_time: {duration:.2f}s")

    def on_train_end(self, logs=None):
        total_duration = time.perf_counter() - self.total_start_time
        median_time = statistics.median(self.epoch_times)

        print("\n" + "="*30)
        print(f"Median time per epoch: {median_time:.4f}s")
        print(f"Total training time: {total_duration:.2f}s")
        print("="*30)

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
time_callback = TimeHistory()
vae.fit(mnist_digits, epochs=100, batch_size=128, callbacks=[time_callback])

### Cuda Implementation

In [ ]:
!chmod +x scripts/run_macro_bench.sh
!bash scripts/run_macro_bench.sh

In [ ]:
!chmod +x scripts/run_nsys_profiling.sh
!bash scripts/run_nsys_profiling.sh

### Kernel Fusion + Tensore Cores

In [ ]:
!chmod +x scripts/run_macro_bench_fused_tc.sh
!bash scripts/run_macro_bench_fused_tc.sh